## Step 1: Connecting to sqlite3 server and creating table(s) within database

In [37]:
import sqlite3
sqlite3.connect('kinase__phospositeDB.db') # Define connection to sqlite3 server

conn = sqlite3.connect('kinase__phospositeDB.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' (python script) file is saved

# To make database, make the tables first

# Create table - KINASE
c.execute('''CREATE TABLE KINASE
             ([uniprot_id] VARCHAR PRIMARY KEY,[Name] VARCHAR, [mass] FLOAT)''')

# Create table - PHOSPHOSITE
c.execute('''CREATE TABLE PHOSPHOSITE
             ([uniprot_id] VARCHAR PRIMARY KEY,[Residue] INTEGER)''')

# Third table to represent the output of a query between the two linked tables from above 
c.execute('''CREATE TABLE KIN_PHOSPHOSITE
             ([Name] VARCHAR, [Residue] INTEGER, [mass] FLOAT)''')


conn.commit()

## Step 2: Import the (raw) Data using Pandas

The imported kinase data will be stored in the KINASE table. 

While the imported Phosphosite data will be stored in the PHOSPHOSITE table. 

We will then be able to link the data from the KINASE table to the data from the PHOSPHOSITE table via the uniprot_ID field.

In [38]:
import pandas as pd
from pandas import DataFrame

conn = sqlite3.connect('kinase__phospositeDB.db')  
c = conn.cursor()

# Importing kinase csv file 
read_kinase = pd.read_csv('BasicKinase.csv')
print(read_kinase)

       Name uniprot_id     mass
0      AKT1     P31749   55.686
1      AKT2     P31751   55.769
2      AKT3     Q9Y243   55.775
3  CDC42BPA     Q5VT25  197.307
4  CDC42BPB     Q9Y5S2  194.315


In [39]:
# Importing phosphosite csv file 
read_phosphosite = pd.read_csv('BasicPhosphosite.csv')
print(read_phosphosite)

  uniprot_id  Residue
0     P31749      450
1     P31751      126
2     Q9Y243      447
3     Q5VT25     1719
4     Q9Y5S2     1682


The two CSVs are successfully imported.
However, currently the tables in the database (KinasePhospositeDB.db) are empty. 
We will extract the data entries in the csv and fill the empty tables 

In [40]:
# Insert the values from the csv file into the table 'KINASE'
read_kinase.to_sql('KINASE', conn, if_exists='append', index = False)

# Replace the values from the csv file into the table 'PHOSPHOSITE'
read_phosphosite.to_sql('PHOSPHOSITE', conn, if_exists='replace', index = False)

## Step 3: Connecting Kinase and Phosphosite tables to make a relational database

In [41]:
c.execute('''
INSERT INTO KIN_PHOSPHOSITE (Name,Residue,mass)
SELECT DISTINCT clt.Name, ctr.Residue, clt.mass 
FROM KINASE clt
LEFT JOIN PHOSPHOSITE ctr ON clt.uniprot_id = ctr.uniprot_id
          ''')

# 'clt' associated to columns from KINASE table and 'ctr' associated to columns from PHOSPHOSITE table

## Step 4: Executing an example SQL query 
The connection has been made. We can now execute an SQL query to get the name of the kinase and its mass for a given residue position of the phosphosite 

In [50]:
# executing an SQL query 
c.execute('''
SELECT DISTINCT *
FROM KIN_PHOSPHOSITE
WHERE Residue = (SELECT max(Residue) FROM KIN_PHOSPHOSITE)
          ''')

# Above SQL command is asking to query based on the highest residue position of the phosphosite and return as the output
# the name of the kinase, its mass 


# Output of the query
print(c.fetchall()) # Returns output as a list

[('CDC42BPA', 1719, 197.30700000000002)]


In [70]:
df = DataFrame(c.fetchall(), columns=['Name','Residue','mass'])

print (df) 

Empty DataFrame
Columns: [Name, Residue, mass]
Index: []


In [67]:
# Insert the values from the INSERT QUERY into the table 'KIN_PHOSPHOSITE'
df.to_sql('KIN_PHOSPHOSITE', conn, if_exists='append', index = False) 